In [1]:
!pip install langchain-community langchain-chroma langchain-openai bs4 > /dev/null

In [2]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.colab import userdata

import bs4

api_key = userdata.get('API_KEY')

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [6]:
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer("section", class_="css-18vt64m")
    )
)
docs = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=api_key)
)

In [8]:
retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: ì‹¶ì�€ ëˆ„êµ¬ë‚˜ì—�ê²Œ ì—´ë\xa0¤ ì�ˆìœ¼ë‹ˆ ì£¼ì\xa0€ë§�ê³\xa0 ë�„ì\xa0„í•´ ë³´ì„¸ìš”.ğŸ’¡<All-in> ì½”ë”© ê³µëª¨ì\xa0„ì—�ì„œ ë§Œë“\xa0 ë‹¤ì–‘í•œ ì„œë¹„ìŠ¤ë¥¼ ë§Œë‚˜ë³´ê³\xa0 ì‹¶ë‹¤ë©´?ë‹¤ì–‘í•œ ì„œë¹„ìŠ¤ì™€ ê¸°ë°œí•œ ì•„ì�´ë””ì–´ê°€ ëª¨ì�¸ ê³³ì—� ì´ˆëŒ€í•©ë‹ˆë‹¤. ì°¸ê°€ì��ë“¤ì�˜ ë¬¸ì\xa0œ í•´ê²°ë°©ë²•ì�´ ê¶�ê¸ˆí•˜ì‹œë‹¤ë©´ ì§€ê¸ˆ ë°”ë¡œ â€˜All-in ê³µëª¨ì\xa0„â€™ì—�ì„œ ë§Œë‚˜ë³´ì„¸ìš”!ğŸ‘‰ğŸ�»Â\xa0ê³µëª¨ì\xa0„ ê²°ê³¼ë¬¼ ë³´ëŸ¬ê°€ê¸°ëˆ„êµ¬ë‚˜ í�°ì�¼ ë‚¼ ìˆ˜ ì�ˆì–´ìŠ¤íŒŒë¥´íƒ€ì½”ë”©í�´ëŸ½ê¸€ | ì‹\xa0ìˆ˜ì§€ íŒ€ìŠ¤íŒŒë¥´íƒ€ ì—�ë””í„°- í•´ë‹¹ ì½˜í…�ì¸\xa0ëŠ” ì\xa0€ì�‘ê¶Œë²•ì—� ì�˜í•´ ë³´í˜¸ë°›ëŠ” ì\xa0€ì�‘ë¬¼ë¡œ ìŠ¤íŒŒë¥´íƒ€ì½”ë”©í�´ëŸ½ì—� ì\xa0€ì�‘ê¶Œì�´ ì�ˆìŠµë‹ˆë‹¤.- í•´ë‹¹ ì½˜

In [9]:
response = llm.invoke(user_prompt)
print(response.content)

ALL-in 코딩 공모전의 수상작들은 다양한 기술 스택을 활용하여 웹 및 모바일 애플리케이션을 개발한 프로젝트들입니다. 수상작들은 프론트엔드에서 React, Spring Boot, Flutter 등 현대적인 프레임워크를 사용했고, 백엔드에서는 Java, MySQL 등 다양한 기술을 접목하여 효율적인 시스템을 구현했습니다. 각 프로젝트는 창의적이고 실용적인 접근 방식을 보여주며, 개별적인 주제와 기능을 가지고 있어 참가자들의 다채로운 아이디어를 반영하였습니다.
